# Universal Approximation in Quantum Circuits

This notebook explores how quantum circuits can approximate arbitrary unitaries, drawing analogies to classical neural networks as universal function approximators.

## Theory

- Quantum circuits can approximate any unitary transformation on $n$ qubits.
- The Solovay–Kitaev theorem guarantees that any unitary can be approximated to arbitrary precision with a universal gate set.
- Variational quantum circuits (VQCs) are parameterized ansätze designed to explore this space of unitaries.

## Analogy to Classical ML

- Neural networks approximate arbitrary **functions**.
- Quantum circuits approximate arbitrary **unitary transformations**.

| Classical ML         | Quantum ML                        |
|-----------------------|-----------------------------------|
| Universal function approximators | Universal unitary approximators |
| Layers / activations  | Parameterized gates / layers      |
| Weights               | Gate parameters                   |

### Example: Single-Qubit Universal Approximation

The below shows any single-qubit unitary can be expressed as RZ-RY-RZ.

In [1]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=1)

# Define a random unitary (composition of rotations)
U_true = qml.matrix(qml.RZ(0.5, 0)) @ qml.matrix(qml.RY(1.2, 0)) @ qml.matrix(qml.RZ(-0.7, 0))

# Decompose into Euler angles (RZ-RY-RZ)
ops = qml.ops.one_qubit_decomposition(U_true, wire=0, return_global_phase=True)
alpha, beta, gamma, _ = [op.data[0] for op in ops]

print(f"Decomposed angles: alpha={alpha:.3f}, beta={beta:.3f}, gamma={gamma:.3f}")

# Reconstruct the unitary
U_reconstructed = (
    qml.matrix(qml.RZ(gamma, 0)) @
    qml.matrix(qml.RY(beta, 0)) @
    qml.matrix(qml.RZ(alpha, 0))
)

print("Original U:\n", U_true)
print("Reconstructed U:\n", U_reconstructed)
print("Are they close?", np.allclose(U_true, U_reconstructed))

Decomposed angles: alpha=11.866, beta=1.200, gamma=0.500
Original U:
 [[ 0.82121237+0.08239607j -0.46601954+0.31882112j]
 [ 0.46601954+0.31882112j  0.82121237-0.08239607j]]
Reconstructed U:
 [[ 0.82121237+0.08239607j -0.46601954+0.31882112j]
 [ 0.46601954+0.31882112j  0.82121237-0.08239607j]]
Are they close? True


## Variational Ansatz

Variational Quantum Circuits (VQCs) use parameterized gates to approximate target unitaries or states.

If the ansatz is sufficiently expressive (deep, wide enough), it can in theory represent any target state.

But in practice:

- Hardware noise limits depth.
- Optimization can get stuck in barren plateaus.

Note:
- In quantum circuits, the "barren plateau" is basically the quantum analog of vanishing gradients in deep learning.

Any unitary can be approximated with enough gates and qubits.
- Variational circuits are quantum analogs of neural networks.
- Expressibility vs. trainability trade-off exists.
- Compilation reduces ideal circuits to hardware-native gates.

## VQE: A Hybrid Quantum-Classical Algorithm

- **VQE (Variational Quantum Eigensolver)** is not a fully quantum algorithm, but a *hybrid* one.
- It combines quantum circuits and classical optimization:
  - The **quantum part** prepares a parameterized state |ψ(θ)⟩ and measures expectation values (like energy ⟨ψ(θ)|H|ψ(θ)⟩).
  - The **classical part** uses optimizers (e.g., gradient descent) to update θ and minimize the cost function.
- The optimization loop is *purely classical*, even though it operates over quantum measurements.
- This approach allows quantum computers to handle *hard-to-simulate* state preparation and measurement, while classical computers handle optimization.

**Key comparison:**
- Unlike purely combinatorial quantum algorithms (like QAOA), which seek discrete solutions (bitstrings), VQE solves a *continuous* eigenvalue problem.
- Both use parameterized quantum circuits and can suffer from *barren plateaus* (vanishing gradients with large circuits).

**Bottom line:**  
> VQE uses quantum power to evaluate hard quantum states, but relies on classical algorithms to *learn* optimal parameters.


## Why is VQE an Eigenvalue Problem?

In quantum chemistry and physics, many problems reduce to finding **the ground state energy** of a system. Mathematically, this is expressed as finding the **smallest eigenvalue** of the Hamiltonian \( H \):

$
H |\psi\rangle = E |\psi\rangle
$

- $H$ is the Hamiltonian operator (Hermitian matrix describing the system).
- $E$ is an eigenvalue (energy).
- $|\psi\rangle$ is the eigenvector (quantum state).

### The Variational Principle

The **variational principle** says:

$
\langle \psi | H | \psi \rangle \geq E_0
$

where $E_0$ is the **true ground-state energy** (smallest eigenvalue).

### How VQE Works

- We don't know the exact eigenvector $|\psi\rangle$.
- Instead, we prepare **parameterized trial states** $|\psi(\theta)\rangle$ on a quantum computer.
- We measure their **expectation value**:

$
\langle \psi(\theta) | H | \psi(\theta) \rangle
$

- By varying **θ**, we **minimize** this expectation value.

### Goal

$
\min_\theta \langle \psi(\theta) | H | \psi(\theta) \rangle \approx E_0
$

- The minimal value approximates the **lowest eigenvalue**.
- The corresponding state approximates the **ground-state eigenvector**.

### Bottom Line

**VQE is solving an eigenvalue problem:**

> *“Find the smallest eigenvalue (ground state energy) of the Hamiltonian $H$.”*

This is why VQE is a hybrid quantum-classical algorithm designed specifically for **eigenvalue problems** in quantum systems.
